In [304]:
# imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import datetime
# see all columns
pd.set_option('display.max_columns', None)
# see all rows
pd.set_option('display.max_rows', None)
# set seaborn style
sns.set_style('whitegrid')
# set context to notebook
sns.set_context('notebook')


In [305]:
# import data

# set path
path = '../data/raw/DonkeyRepublic/'

hubs = pd.read_excel(path + 'Hubs_2019-4-2_1201.xlsx', parse_dates=['created_at'])
rentals = pd.read_excel(path + 'Rentals_2019-4-2_1456.xlsx', parse_dates=['created_at', 'finished_at'])

In [306]:
# check data
hubs.head()


,created_at,latitude,longitude,id,name,deleted_at
0,2018-05-21 09:35:00.697175,48.864936,2.310624,3268,Cours la Reine,2018-05-21 16:17:24.722283
1,2018-11-26 09:06:05.590590,55.695252,12.547185,6367,Heinesgade,NaN
2,2018-06-05 12:12:28.639837,55.676916,12.564896,3642,Concert Hall Pumpehuset,2018-11-15 10:45:34.468962
3,2018-05-28 20:07:34.173984,55.668700,12.551114,3526,Saxogade,NaN
4,2018-11-08 09:36:50.440822,55.699557,12.515065,6233,GrÃ¸ndal Multicenter,NaN


In [307]:
hubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266 entries, 0 to 1265
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   created_at  1266 non-null   datetime64[ns]
 1   latitude    1266 non-null   float64       
 2   longitude   1266 non-null   float64       
 3   id          1266 non-null   int64         
 4   name        1266 non-null   object        
 5   deleted_at  119 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 59.5+ KB


In [308]:
# remove deleted_at column
hubs.drop('deleted_at', axis=1, inplace=True)

In [309]:
hubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266 entries, 0 to 1265
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   created_at  1266 non-null   datetime64[ns]
 1   latitude    1266 non-null   float64       
 2   longitude   1266 non-null   float64       
 3   id          1266 non-null   int64         
 4   name        1266 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 49.6+ KB


In [310]:
rentals.head()

,created_at,finished_at,pickup_hub_id,dropoff_hub_id,user_id
0,2018-03-01 17:43:14.707445,2018-03-01 18:14:12.145,2163,2449.0,108186
1,2018-03-02 09:55:18.823405,2018-03-02 16:56:35.013,2381,2381.0,113852
2,2018-03-02 14:00:10.755516,2018-03-02 17:13:08.047,1513,1513.0,113912
3,2018-03-01 10:25:34.429934,2018-03-02 20:23:46.544,2337,2337.0,113822
4,2018-03-02 08:51:47.459257,2018-03-02 21:18:48.813,2153,233.0,113881


In [311]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279860 entries, 0 to 279859
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   created_at      279860 non-null  object 
 1   finished_at     279408 non-null  object 
 2   pickup_hub_id   279860 non-null  int64  
 3   dropoff_hub_id  261293 non-null  float64
 4   user_id         279860 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 10.7+ MB


In [312]:
# print unique values for id in hubs and for pickup_hub_id and dropoff_hub_id in rentals
print('Hubs id unique values: ', hubs.id.nunique())
print('Pickup hub id unique values: ', rentals.pickup_hub_id.nunique())
print('Dropoff hub id unique values: ', rentals.dropoff_hub_id.nunique())

Hubs id unique values:  1266
Pickup hub id unique values:  1244
Dropoff hub id unique values:  1227


In [313]:
# count nr of nan values in dropoff_hub_id
print('Nr of nan values in dropoff_hub_id: ', rentals.dropoff_hub_id.isna().sum())

Nr of nan values in dropoff_hub_id:  18567


In [314]:
# print dataframe head for rentals with nan values in dropoff_hub_id
rentals[rentals.dropoff_hub_id.isna()].head()

,created_at,finished_at,pickup_hub_id,dropoff_hub_id,user_id
24,2018-03-03 11:50:26.336953,2018-03-05 09:54:43.594933,2251,NaN,113998
47,2018-03-07 15:07:37.999381,2018-03-08 09:14:28.575214,2281,NaN,114435
62,2018-03-10 11:00:33.847099,2018-03-10 13:54:22.669873,2227,NaN,27688
118,2018-03-09 15:30:12.161583,2018-03-12 10:00:32.844345,152,NaN,16
132,2018-03-12 10:18:23.874653,2018-03-12 15:45:00.200976,2337,NaN,115368


In [315]:
# see the proportion of nan values in dropoff_hub_id
print('Proportion of nan values in dropoff_hub_id: ', rentals.dropoff_hub_id.isna().sum() / len(rentals))

Proportion of nan values in dropoff_hub_id:  0.0663438862288287


In [316]:
# drop rows with nan values in rentals

rentals.dropna(inplace=True)

In [317]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 261189 entries, 0 to 279859
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   created_at      261189 non-null  object 
 1   finished_at     261189 non-null  object 
 2   pickup_hub_id   261189 non-null  int64  
 3   dropoff_hub_id  261189 non-null  float64
 4   user_id         261189 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 12.0+ MB


In [318]:
# see rentals time interval
print('Rentals time interval: ', rentals.created_at.min(), rentals.created_at.max())

Rentals time interval:  2018-03-01 08:12:40.421178 2019-04-02 12:39:57.98495


In [319]:
hubs.head()

,created_at,latitude,longitude,id,name
0,2018-05-21 09:35:00.697175,48.864936,2.310624,3268,Cours la Reine
1,2018-11-26 09:06:05.590590,55.695252,12.547185,6367,Heinesgade
2,2018-06-05 12:12:28.639837,55.676916,12.564896,3642,Concert Hall Pumpehuset
3,2018-05-28 20:07:34.173984,55.668700,12.551114,3526,Saxogade
4,2018-11-08 09:36:50.440822,55.699557,12.515065,6233,GrÃ¸ndal Multicenter


In [320]:
# in hubs explore names that contain non english characters
hubs[hubs.name.str.contains('[^a-zA-Z0-9\s]')].head()

,created_at,latitude,longitude,id,name
4,2018-11-08 09:36:50.440822,55.699557,12.515065,6233,GrÃ¸ndal Multicenter
7,2019-03-14 11:39:00.814359,55.697218,12.584605,7148,SÃ¸nderborggade
8,2018-07-22 15:03:16.438551,55.669671,12.545823,4947,MatthÃ¦usgade
14,2018-11-29 18:53:59.393007,55.616528,12.585753,6411,PilegÃ¥rd Alle
17,2018-12-03 12:31:53.789002,55.637874,12.590540,6434,Hf. Elmebo


In [321]:
# replace Ã¦ with æ in hubs names
# replace Ã¸Ã˜ with ø, Ø in hubs names
# replace Ã¥ with å in hubs names
# replace Ã… with Å in hubs names
# replace Ã© with é in hubs names
# replace Ã¼ with ü in hubs names
# replace Ã¶ with ö in hubs names
# replace non-ascii characters in hubs names
hubs['name'] = hubs['name'].str.replace('Ã¦', 'æ', regex=True)
hubs['name'] = hubs['name'].str.replace('Ã¸', 'ø', regex=True)
hubs['name'] = hubs['name'].str.replace('Ã¥', 'å', regex=True)
hubs['name'] = hubs['name'].str.replace('Ã…', 'Å', regex=True)
hubs['name'] = hubs['name'].str.replace('Ã˜', 'Ø', regex=True)
hubs['name'] = hubs['name'].str.replace('Ã©', 'é', regex=True)
hubs['name'] = hubs['name'].str.replace('Ã¼', 'ü', regex=True)
hubs['name'] = hubs['name'].str.replace('Ã¶', 'ö', regex=True)

# remove entire substring if it is in between [] like [N], [G]...
hubs['name'] = hubs['name'].str.replace('\[[^\]]*\]', '', regex=True)

In [322]:
hubs.head()

,created_at,latitude,longitude,id,name
0,2018-05-21 09:35:00.697175,48.864936,2.310624,3268,Cours la Reine
1,2018-11-26 09:06:05.590590,55.695252,12.547185,6367,Heinesgade
2,2018-06-05 12:12:28.639837,55.676916,12.564896,3642,Concert Hall Pumpehuset
3,2018-05-28 20:07:34.173984,55.668700,12.551114,3526,Saxogade
4,2018-11-08 09:36:50.440822,55.699557,12.515065,6233,Grøndal Multicenter


In [323]:
# check how man dropoff_hub_id are not in hubs id
print('Nr of dropoff_hub_id not in hubs id: ', rentals[~rentals.dropoff_hub_id.isin(hubs.id)].dropoff_hub_id.nunique())

Nr of dropoff_hub_id not in hubs id:  36


In [324]:
# merge hubs and rentals on pickup_hub_id
rentals = rentals.merge(hubs[['id', 'latitude', 'longitude', 'name']], how='left', left_on='pickup_hub_id', right_on='id', suffixes=('', 'Start'))
rentals.head()

,created_at,finished_at,pickup_hub_id,dropoff_hub_id,user_id,id,latitude,longitude,name
0,2018-03-01 17:43:14.707445,2018-03-01 18:14:12.145,2163,2449.0,108186,2163.0,55.673440,12.564409,Central Station
1,2018-03-02 09:55:18.823405,2018-03-02 16:56:35.013,2381,2381.0,113852,2381.0,55.688937,12.562486,Ravnsborggade
2,2018-03-02 14:00:10.755516,2018-03-02 17:13:08.047,1513,1513.0,113912,1513.0,55.682558,12.580462,Møntergade
3,2018-03-01 10:25:34.429934,2018-03-02 20:23:46.544,2337,2337.0,113822,2337.0,55.670289,12.565058,København H - Bus Stops
4,2018-03-02 08:51:47.459257,2018-03-02 21:18:48.813,2153,233.0,113881,2153.0,55.680517,12.587455,Nyhavn


In [325]:
# for each unique id in dropoff_hub_id, get the corresponding latitude, longitude and name from hubs
# add these columns to rentals named LatitudeEnd, LongitudeEnd, NameEnd
rentals = rentals.merge(hubs[['id', 'latitude', 'longitude', 'name']], how='left', left_on='dropoff_hub_id', right_on='id', suffixes=('', 'End'))

In [326]:
rentals.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261189 entries, 0 to 261188
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   created_at      261189 non-null  object 
 1   finished_at     261189 non-null  object 
 2   pickup_hub_id   261189 non-null  int64  
 3   dropoff_hub_id  261189 non-null  float64
 4   user_id         261189 non-null  int64  
 5   id              258232 non-null  float64
 6   latitude        258232 non-null  float64
 7   longitude       258232 non-null  float64
 8   name            258232 non-null  object 
 9   idEnd           258209 non-null  float64
 10  latitudeEnd     258209 non-null  float64
 11  longitudeEnd    258209 non-null  float64
 12  nameEnd         258209 non-null  object 
dtypes: float64(7), int64(2), object(4)
memory usage: 25.9+ MB


In [327]:
# drop nan values in rentals
rentals.dropna(inplace=True)

In [328]:
# rename columns, created_at to StartTime, finished_at to EndTime, name to PickupHubName, latitude to LatitudeStart, longitude to LongitudeStart
rentals.rename(columns={'created_at': 'StartTime', 'finished_at': 'EndTime', 'latitude': 'latitudeStart', 
                        'longitude': 'longitudeStart', "pickup_hub_id": "StartHubId", "dropoff_hub_id": "EndHubId",
                        "user_id": "UserId", "name":"nameStart"}, inplace=True)

In [329]:
rentals.head()

,StartTime,EndTime,StartHubId,EndHubId,UserId,id,latitudeStart,longitudeStart,nameStart,idEnd,latitudeEnd,longitudeEnd,nameEnd
0,2018-03-01 17:43:14.707445,2018-03-01 18:14:12.145,2163,2449.0,108186,2163.0,55.673440,12.564409,Central Station,2449.0,55.658239,12.605434,Skotlands Plads
1,2018-03-02 09:55:18.823405,2018-03-02 16:56:35.013,2381,2381.0,113852,2381.0,55.688937,12.562486,Ravnsborggade,2381.0,55.688937,12.562486,Ravnsborggade
2,2018-03-02 14:00:10.755516,2018-03-02 17:13:08.047,1513,1513.0,113912,1513.0,55.682558,12.580462,Møntergade,1513.0,55.682558,12.580462,Møntergade
3,2018-03-01 10:25:34.429934,2018-03-02 20:23:46.544,2337,2337.0,113822,2337.0,55.670289,12.565058,København H - Bus Stops,2337.0,55.670289,12.565058,København H - Bus Stops
4,2018-03-02 08:51:47.459257,2018-03-02 21:18:48.813,2153,233.0,113881,2153.0,55.680517,12.587455,Nyhavn,233.0,55.668475,12.557384,Høkerboderne


In [330]:
# drop id and idEnd columns
rentals.drop(['id', 'idEnd'], axis=1, inplace=True)

In [331]:
rentals.info()  

<class 'pandas.core.frame.DataFrame'>
Index: 255990 entries, 0 to 261188
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   StartTime       255990 non-null  object 
 1   EndTime         255990 non-null  object 
 2   StartHubId      255990 non-null  int64  
 3   EndHubId        255990 non-null  float64
 4   UserId          255990 non-null  int64  
 5   latitudeStart   255990 non-null  float64
 6   longitudeStart  255990 non-null  float64
 7   nameStart       255990 non-null  object 
 8   latitudeEnd     255990 non-null  float64
 9   longitudeEnd    255990 non-null  float64
 10  nameEnd         255990 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 23.4+ MB


In [335]:
# set StartTime and EndTime to datetime
rentals['StartTime'] = pd.to_datetime(rentals['StartTime'], format="mixed")
rentals['EndTime'] = pd.to_datetime(rentals['EndTime'], format="mixed")
# set EndHubId to int
rentals['EndHubId'] = rentals['EndHubId'].astype(int)

In [336]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255990 entries, 0 to 261188
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   StartTime       255990 non-null  datetime64[ns]
 1   EndTime         255990 non-null  datetime64[ns]
 2   StartHubId      255990 non-null  int64         
 3   EndHubId        255990 non-null  int32         
 4   UserId          255990 non-null  int64         
 5   latitudeStart   255990 non-null  float64       
 6   longitudeStart  255990 non-null  float64       
 7   nameStart       255990 non-null  object        
 8   latitudeEnd     255990 non-null  float64       
 9   longitudeEnd    255990 non-null  float64       
 10  nameEnd         255990 non-null  object        
dtypes: datetime64[ns](2), float64(4), int32(1), int64(2), object(2)
memory usage: 22.5+ MB


In [338]:
duplicate_rows_new = rentals.duplicated().sum()

In [339]:
duplicate_rows_new

207

In [340]:
# drop duplicates
rentals.drop_duplicates(inplace=True)

In [341]:
# save rentals to csv
rentals.to_csv('../data/processed/donkey_rentals.csv', index=False)